In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import peak_widths
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
%matplotlib inline

In [ ]:
def read_raman_data(file_path):
    """
    Reads Raman data from a text file.

    Inputs:
    - file_path: Path to the text file containing Raman data.

    Returns:
    - raman_shift: Array containing Raman shift values.
    - intensity: Array containing respective intensity values.
    - peak_threshold: Value calculated by average intensity added by some integer
    """
    data = np.loadtxt(file_path)
    raman_shift = data[:,0]
    intensity = data[:,1] # for map txt files, change the integer to match the column of a plot you'd like to visualize
    return raman_shift, intensity

In [ ]:
def index_to_xdata(xdata, indices):
    "interpolate the values from signal.peak_widths to proper raman_shift data"
    ind = np.arange(len(xdata))
    f = interp1d(ind,xdata)
    return f(indices)

In [ ]:
def plot_raman_spectrum(raman_shift, intensity, peaks, labels, ratio_label = None):
    """
    Plots the Raman spectrum, AND calculates integrals underneath peaks.

    Inputs:
    - raman_shift: Array containing Raman shift values.
    - intensity: Array containing Respective intensity values.
    - peaks: Dictionary containing peaks data, if it exists.
    - labels: User created list containing labels, left to right, of expected peaks

    Returns:
    - integrals: Array containing peaks' respective integral values.
    
    """
    plt.figure(figsize=(11, 6)) #adjust figure size as needed
    plt.plot(raman_shift, intensity, color='red') #change color as desired
    
    peak_values = peaks['peak_intensity']
    if len(peak_values) != len(labels):
        raise ValueError(f"Length of labels list does not match amount of peaks present: {len(peak_values)}")
    peak_ratio = peaks['peak_ratio']
    widths = peaks['peak_widths']
    width_heights = peaks['width_heights']
    left_ips = peaks['left_ips']
    right_ips = peaks['right_ips']
    peak_raman_shifts = peaks['peak_raman_shifts']
    integrals = []
    plt.scatter(peak_raman_shifts, peak_values, color='blue', marker='o')
    for i, (shift, value, width, label, left_ip, right_ip) in enumerate(zip(peak_raman_shifts, peak_values, widths, labels, left_ips, right_ips)):
        plt.text(shift + 100, value, label, ha='center', va='bottom', color='blue')
        plt.text(shift + 100, value/2, f"W:{math.ceil(width)}", ha='center', va='bottom', color='black') #adjust shift and/or value as needed
        plt.axvline(shift, color='black', linestyle='--', linewidth=1)
        plt.text(shift- 100, value/40, f"Sh:{math.ceil(shift)}", ha='center', va='bottom', color='black')
        plt.scatter(left_ip, value, color = 'orange', marker = 'v') # perhaps use this and the next lines in order to define bounds of integration
        plt.scatter(right_ip, value, color = 'orange', marker = 'v')
        plt.fill_between(raman_shift,intensity, 0, where=(raman_shift>=left_ip)&(raman_shift<=right_ip),color = 'blue', alpha = 0.25)
        integral = np.trapz(intensity[(raman_shift >= left_ip) & (raman_shift <= right_ip)], 
                            raman_shift[(raman_shift >= left_ip) & (raman_shift <= right_ip)])
        print(left_ip, right_ip)
        if integral not in integrals:
            integrals.append(integral)
        else:
            raise ValueError("integral exists in integrals already")
            
    plt.hlines(width_heights, left_ips, right_ips, color='g')
    if peak_ratio:
        plt.annotate(f"{ratio_label} ratio is {peak_ratio}", xy=(1900, np.mean(peak_values)), xytext=(1700, np.mean(peak_values))) #adjust xy and xytext as needed
    plt.ylim(0, max(intensity)+50)
    plt.xlim(min(raman_shift), max(raman_shift))    
    plt.xlabel('Raman Shift (cm^-1)')
    plt.ylabel('Intensity (a.u.)')
    plt.title("Material") #For whatever material you're working with
    plt.grid(True)
    plt.show()
    return integrals

In [ ]:
def identify_peaks(raman_shift, intensity, desired_ratio, threshold, distance, fwhm):
    """
    Identifies peaks in the Raman spectrum using scipy's find_peaks function.
    Identifies peak widths in the Raman spectrum using scipy's peak_widths function. 

    Inputs:
    - raman_shift: Array containing Raman shift values.
    - intensity: Array containing corresponding intensity values.
    - threshold: Minimum intensity required to be considered as a peak.
    - distance: Minimum distance between peaks (in terms of array indices).

    Returns:
    - peaks: Dictionary containing peak indices, values, peak ratio, raman shift peak locations, width values per peak, width heights,
             and raman shift locations for widths
    """
    peaks, _ = find_peaks(intensity, height=threshold, distance=distance) 
    widths, width_heights, left_ips, right_ips = peak_widths(intensity, peaks, rel_height = fwhm) #adjust rel_height to find peak width at specific y vals 
    left_ips = index_to_xdata(raman_shift, left_ips) 
    right_ips = index_to_xdata(raman_shift, right_ips) 
    peak_intensity = intensity[peaks]
    peak_raman_shifts = raman_shift[peaks]
    if len(peaks) == 0:
        raise ValueError("Your input for peak_threshold was not able to locate any peaks. Please adjust.")
    if (len(peaks) >= 2) and ((2 and 1) in desired_ratio):
        if len(desired_ratio) != len(peaks):
            raise ValueError(f"There are {len(peaks)} peaks and you put in {len(desired_ratio)} indices in desired_ratio")
        large = None
        small = None
        for i, j in enumerate(desired_ratio):
            if j == 2:
                large = peak_intensity[i]
            elif j == 1:
                small = peak_intensity[i]
            else:
                pass
        peak_ratio = large/small
    else:
        peak_ratio = None
    return {'peak_indices': peaks, 'peak_intensity': peak_intensity, 'peak_ratio': peak_ratio, 
            'peak_raman_shifts': peak_raman_shifts, 'peak_widths': widths, 'width_heights': width_heights, 'left_ips': left_ips,
            'right_ips': right_ips}

In [ ]:
def peak_integral_ratio(desired_ratio, integrals):
    """
    Calculates desired peak integral ratio. 

    Inputs:
    - desired_ratio: Array containing user inputted integers ranging from 0-2.
    - integrals: Array containing peaks' corresponding integral values.

    Returns:
    - integral_ratio: Ratio between selected peak integrals.
    """
    if len(desired_ratio) != len(integrals):
        raise ValueError(f"There are {len(integrals)} integrals and you put in {len(desired_ratio)} indices in desired_ratio")
    large = None
    small = None
    for i, j in enumerate(desired_ratio):
        if j == 2:
            large = integrals[i]
        elif j == 1:
            small = integrals[i]
        else:
            pass
    integral_ratio = large/small
    return integral_ratio

In [ ]:
def RatioLabeller(labels, desired_ratio):
    """
    Labels calculated ratio. 

    Inputs:
    - labels: Array containing user inputted strings. 
    - desired_ratio: Array containing user inputted integers ranging from 0-2.

    Returns:
    - ratio_label: Label of ratio found from labels and desired ratio.
    """
    top = None
    bottom = None
    for i, j in enumerate(desired_ratio):
        if j == 2:
            top = labels[i]
        elif j == 1:
            bottom = labels[i]
        else:
            pass
    ratio_label = f"{top}/{bottom}"
    return ratio_label

In [ ]:
file_path = "path/to/file"  # Change this to your Raman data file path
labels = ['1st', '2nd', '3rd']
desired_ratio = [2,1,0] # Input 2 for peak desired for numerator, and 1 for the peak desired in the denominator
ratio_label = RatioLabeller(labels, desired_ratio)
raman_shift, intensity = read_raman_data(file_path) 
peaks = identify_peaks(raman_shift, intensity, desired_ratio, threshold=100, distance=110, fwhm = 0.5)  # Adjust threshold and distance as needed
integrals = plot_raman_spectrum(raman_shift, intensity, peaks, labels, ratio_label)
value = peak_integral_ratio(desired_ratio, integrals)
print(f"Integral ratio for {ratio_label} is {value}")

In [ ]:
peaks['peak_raman_shifts'] # these values actually tell you what the raman shift value is for each respective peak

In [ ]:
peaks['peak_indices'] # these values are extremely important because they highlight where the peaks are in our dataset

In [ ]:
peaks['peak_intensity'] # these values tell us where on the intensity axis the peaks are located

In [ ]:
peaks['peak_ratio'] # this value tells us the peak ratio from Peak1/Peak2

In [ ]:
peaks['peak_widths'] # this list tells us the width values for each peak in the plot, left to right

In [ ]:
integrals # list of integrals, left to right